In [29]:
from langchain_core.tools import BaseTool, ToolException, Field
from langchain.pydantic_v1 import BaseModel
from typing import (
    List, Sequence, Tuple, Callable, Optional, Union, Literal,
    Dict, cast, TYPE_CHECKING, Any, Type, 
)
from io import IOBase
import pandas as pd
import os
from glob import glob

In [31]:
data_path = os.path.join("../data")
os.path.exists(data_path)
csv_files = glob(f"{data_path}/*.csv")
print(csv_files)
lfile = csv_files[1]

['../data/country_eng.csv', '../data/custom_2020.csv', '../data/Mobile-Price-Prediction-cleaned_data.csv']


In [32]:
data_path = os.path.join("../data")
os.path.exists(data_path)
csv_files = glob(f"{data_path}/*.csv")
print(csv_files)
sfile = csv_files[0]
df_small = pd.read_csv(sfile)
print('small csv:\n', df_small.count())
lfile = csv_files[1]
df_large = pd.read_csv(lfile)
print()
print('large csv:\n', df_large.count())

['../data/country_eng.csv', '../data/custom_2020.csv', '../data/Mobile-Price-Prediction-cleaned_data.csv']
small csv:
 Unnamed: 0      232
Country         232
Country_name    232
Area            232
dtype: int64

large csv:
 exp_imp    3299804
Year       3299804
month      3299804
ym         3299804
Country    3299804
Custom     3299804
hs2        3299804
hs4        3299804
hs6        3299804
hs9        3299804
Q1         3299804
Q2         3299804
Value      3299804
dtype: int64


In [90]:
df_large.head()

,exp_imp,Year,month,ym,Country,Custom,hs2,hs4,hs6,hs9,Q1,Q2,Value
0,1,2020,1,202001,103,100,0,0,0,190,0,526875,1150084
1,1,2020,1,202001,105,100,0,0,0,190,0,1241751,3830662
2,1,2020,1,202001,106,100,0,0,0,190,0,505391,1530045
3,1,2020,1,202001,107,100,0,0,0,190,0,1852,5150
4,1,2020,1,202001,108,100,0,0,0,190,0,455623,5664430


In [34]:
df_large[df_large["Country"] == 103]["Value"].sum()

6854247392

In [51]:
class CSVToolSchema(BaseModel):
    path: str = Field(description="path of the csv file or a list of paths")
    query: str = Field(description="query for the pandas dataframe")
    max_row_out: Optional[int] = 100
    

In [79]:
class CSVTool(BaseTool):
    name = "csv_tool"
    description = ""
    args_schema: Type[BaseModel] = CSVToolSchema
    return_direct: bool = True
    handle_tool_error: Optional[Union[bool, str, Callable[[ToolException], str]]] = True
    response_format: Literal['content', 'content_and_artifact'] = 'content_and_artifact'
    
    def _run(self, path: Union[str, List[str]], query: str, 
             max_row_out: Optional[int]=100, **_kwargs):
        try:
            # Ensure pandas is available
            import pandas as pd
        except ImportError:
            raise ImportError(
                "pandas package not found, please install with `pip install pandas`."
            )
        
        # Load the CSV into a DataFrame
        if isinstance(path, (str, IOBase)):
            df = pd.read_csv(path, **_kwargs)
        elif isinstance(path, list):
            df = pd.concat([pd.read_csv(item, **_kwargs) for item in path], ignore_index=True)
        else:
            raise ValueError(f"Expected str, list, or file-like object, got {type(path)}")
        
        # Safely evaluate the query string on the DataFrame
        artifact = {} ### should be a dictionary 
        try:
            result = eval(query,  {"df": df, "pd": pd, "__builtins__": None})
            if isinstance(result, (pd.core.frame.DataFrame, 
                                   pd.core.series.Series,
                                   List, tuple)):
                if isinstance(result, pd.DataFrame) and result.empty:
                    content = "Query result is an empty DataFrame."
                else:
                    content = f"Result Size: {result.size}. Examples: {result.head(max_row_out)}"
                artifact['out_df'] = result
            else:
                content = str(result)
        except Exception as e:
            raise ValueError(f"Failed to execute query: {e}")        
        return content, artifact

# Example usage
csv_tool = CSVTool()
path_to_csv = lfile
query_string = 'df[df["Country"] == 103]["Value"].sum()'  # Example query string
result = csv_tool._run(path=path_to_csv, query=query_string)
print(result)
result = csv_tool.invoke({'path': path_to_csv, 'query': query_string})
print(result)
query_string = 'df[df["Value"] < 1000]'
result = csv_tool.invoke({'path': path_to_csv, 'query': query_string})
print(result)

('6854247392', {})
6854247392
Result Size: 13172289. Examples:      exp_imp  Year  month      ym  Country  Custom  hs2  hs4    hs6       hs9  \
9          1  2020      1  202001      116     100    0    0      0       190   
21         1  2020      1  202001      132     100    0    0      0       190   
22         1  2020      1  202001      134     100    0    0      0       190   
31         1  2020      1  202001      149     100    0    0      0       190   
39         1  2020      1  202001      209     100    0    0      0       190   
..       ...   ...    ...     ...      ...     ...  ...  ...    ...       ...   
272        1  2020      1  202001      113     100    7  714  71430  71430000   
274        1  2020      1  202001      601     100    7  714  71430  71430000   
275        1  2020      1  202001      205     100    7  714  71490  71490000   
276        1  2020      1  202001      106     100    8  805  80521  80521000   
278        1  2020      1  202001      112    

In [80]:
tool_message = csv_tool.invoke(
    {
        "name": "csv_tool",
        "args": {'path': path_to_csv, 'query': query_string},
        "id": "1",  # required field
        "type": "tool_call",  # required field
    }
)
print(tool_message)

content='Result Size: 13172289. Examples:      exp_imp  Year  month      ym  Country  Custom  hs2  hs4    hs6       hs9  \\\n9          1  2020      1  202001      116     100    0    0      0       190   \n21         1  2020      1  202001      132     100    0    0      0       190   \n22         1  2020      1  202001      134     100    0    0      0       190   \n31         1  2020      1  202001      149     100    0    0      0       190   \n39         1  2020      1  202001      209     100    0    0      0       190   \n..       ...   ...    ...     ...      ...     ...  ...  ...    ...       ...   \n272        1  2020      1  202001      113     100    7  714  71430  71430000   \n274        1  2020      1  202001      601     100    7  714  71430  71430000   \n275        1  2020      1  202001      205     100    7  714  71490  71490000   \n276        1  2020      1  202001      106     100    8  805  80521  80521000   \n278        1  2020      1  202001      112     100    8

In [77]:
### any other command except df and pd should not run 
try:
    result = csv_tool.invoke({'path': path_to_csv, 'query': 'import os, os.listdir()'})
    print(result)
except Exception as e:
    print(e)

Failed to execute query: invalid syntax (<string>, line 1)


In [78]:
### any other command except df and pd should not run 
try:
    result = csv_tool.invoke({'path': path_to_csv, 'query': "print(__import__('os').listdir())"})
    print(result)
except Exception as e:
    print(e)

Failed to execute query: 'NoneType' object is not subscriptable
